In [ ]:
import numpy as np
from copy import deepcopy

# Part 1

In [ ]:
test = ['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.',
        'Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.'] 

In [ ]:
def get_blueprints(data, part=1):
    blueprint = {}
    
    if part == 2 and len(data)>3:
        data = data[:3]
    
    for line in data:
        line = line.strip().split('.')
        
        bp, line[0] = line[0].split(':')
        bp = bp.split()
        bp = int(bp[1])
        
        ore_bp = line[0].split()
        ore = [int(ore_bp[4]), 0, 0]
        
        clay_bp = line[1].split()
        clay = [int(clay_bp[4]), 0, 0]
        
        obsi_bp = line[2].split()
        obsi = [int(obsi_bp[4]), int(obsi_bp[7]), 0]
        
        geod_bp = line[3].split()
        geod = [int(geod_bp[4]), 0, int(geod_bp[7])]
        
        blueprint[bp] = [ore, clay, obsi, geod]
        
    return blueprint

def get_dt(blueprint, bots, mats):
    dt = 0
    for i in range(0, 3):
        if bots[i] == 0:
            continue
        if mats[i] == blueprint[i]:
            m_dt = 0
        else:
            m_dt = int(np.ceil((blueprint[i]-mats[i])/bots[i]))
        if m_dt > dt:
            dt = m_dt
            
    return dt

def future(score, bots, mats, t, max_time):
    dt = max_time-t
    extra = bots[3]*dt
    extra += int((dt*(dt+1))/2)
    
    if mats[3]+extra < max(score):
        return False
    return True

def DFS(score, blueprint, bots, mats, t, max_bots, max_time, next_bot):
    #wait for mats
    dt = get_dt(blueprint[next_bot], bots, mats)
    
    if t+dt > max_time:
        dt = max_time-t
        mats[3] += bots[3]*dt
        score.append(mats[3])
        return
    
    for i in range(0, 4):
        mats[i] += bots[i]*dt
    t += dt
        
    #start build
    t += 1
    for i in range(0, 3):
        mats[i] -= blueprint[next_bot][i]
    #dig
    for i in range(0, 4):
        mats[i] += bots[i]
    #end build
    bots[next_bot] += 1
    
    if t == max_time:
        score.append(mats[3])
        return
    
    if len(score)>0 and future(score, bots, mats, t, max_time) == False:
        dt = max_time-t
        mats[3] += bots[3]*dt
        score.append(mats[3])
        return
    
    #choose bot
    poss_bots = []
    if bots[0] < max_bots[0]:
        poss_bots.append(0)
    if bots[1] < max_bots[1]:
        poss_bots.append(1)
    if bots[1] > 0 and bots[2] < max_bots[2]:
        poss_bots.append(2)
    if bots[2] > 0 and bots[3] < max_bots[3]:
        poss_bots.append(3)
        
    if len(poss_bots) == 0:
        dt = max_time-t
        mats[3] += bots[3]*dt
        score.append(mats[3])
        return
    
    for pb in poss_bots:
        DFS(score, blueprint, deepcopy(bots), deepcopy(mats), deepcopy(t), max_bots, max_time, pb)
    
def get_geode(data, max_time, part=1, fitst_score=0):
    blueprints = get_blueprints(data, part)
    
    best = []
    scores = []
    for key in blueprints.keys():
        #print(key)
        blueprint = blueprints[key]
        bots = [1,0,0,0]
        mats = [0,0,0,0]
        t = 0
        max_bots = [max([blueprint[0][0],blueprint[1][0],blueprint[2][0],blueprint[3][0]]),
                    max([blueprint[0][1],blueprint[1][1],blueprint[2][1],blueprint[3][1]]),
                    max([blueprint[0][2],blueprint[1][2],blueprint[2][2],blueprint[3][2]]),
                    max_time]
        
        score = [fitst_score]
        for i in range(0, 2):
            #print(key, i)
            DFS(score, blueprint, deepcopy(bots), deepcopy(mats), deepcopy(t), max_bots, max_time, i)
        print(key, max(score))
        scores.append(max(score)*key)
        best.append(max(score))
        #break
    if part == 1:
        print('Part 1 result:', sum(scores))
    elif part == 2:
        print('Part 2 result:', np.prod(best))
    
get_geode(test, 24)

In [ ]:
with open('day19_input.txt', 'r') as f:
    inpt = f.readlines()
    f.close()
get_geode(inpt, 24)

# Part 2

In [ ]:
get_geode(test, 32, 33)

In [ ]:
get_geode(inpt, 32, 1528)